In [1]:
import pandas as pd
symbols = pd.read_csv('nasdaq-100-symbols.csv', sep = ',', header = None)

In [2]:
from datetime import timedelta, date
import math
import pandas as pd
import yfinance as yf
from sympy.solvers import solve
from sympy import Symbol

In [32]:
def ticker(symbol):
    t = yf.Ticker(symbol)
    return t

# h = ticker(symbol)
# h.info

In [4]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [5]:
def tickerInformation(symbol):
    h = ticker(symbol).history(start = date.today() - timedelta(days = 5), end = date.today())
    
    h['diff'] = h['Close'].diff(periods = 1)
    closingprice = h['Close'][-1]
    volume = h['Volume'][-1]
    
    if any(h['diff'] > 0) == False:
        advice = True
    else:
        advice = False
    
    return advice, closingprice, volume

In [6]:
def inputNumber(message):
    while True:
        try:
            userInput = int(input(message))       
        except ValueError:
            print("Not an integer! Try again.")
            continue
        else:
            return userInput 
            break 

In [7]:
from forex_python.converter import CurrencyRates
c = CurrencyRates()
fxRate = c.get_rate('USD', 'NOK')

In [ ]:
df = pd.DataFrame()

columns = ['i', 'symbol', 'advice', 'price', 'price_nok', 'volume']
data = []

print('Generating advice for: {} \n'.format(date.today()))

i = 1
for symbol in  symbols[0]:

    advice, price, volume  = tickerInformation(symbol)

    
    if advice == True:
        
        print('{}. {} advice is buy: {} at {}, with volume {}'.format(i, symbol, advice, price, volume))
        
        price_nok = price * fxRate    
        values = [i, symbol, advice, price, price_nok, volume]
        zipped = dict(zip(columns, values))
        data.append(zipped)
        i = i + 1
        
print('\n Finished generating advice')

df = df.append(data, True)
df

In [8]:
import time
from tqdm import tqdm

df = pd.DataFrame()
columns = ['i','symbol', 'advice', 'price', 'price_nok', 'volume']
data = []

print('Generating advice for: {}'.format(date.today()))
for symbol in  tqdm(symbols[0]):

    advice, price, volume  = tickerInformation(symbol)
    price_nok = price * fxRate
    
    values = [i, symbol, advice, price, price_nok, volume, date.today()]
    zipped = dict(zip(columns, values))
    data.append(zipped)
    
print('Finished generating advice')

df = df.append(data, True)
print(df[df['advice'] == True])

Generating advice for: 2020-05-24


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.84it/s]


Finished generating advice


,i,symbol,advice,price,price_nok,volume
54,55,LBTYA,True,20.84,208.472627,951800
55,56,LBTYK,True,20.49,204.971407,2914000
67,68,NFLX,True,429.32,4294.696163,5419500


In [39]:
import smtplib

content = 'ahoi'

mail = smtplib.SMTP('smtp.gmail.com', 587)

mail.ehlo

mail.starttls()

mail.login('jeroenvanzeeland@gmail.com', 'x45fg21!q')

mail.sendmail('frommail', 'receiver', content)

mail.close()

SMTPAuthenticationError: (534, b'5.7.9 Application-specific password required. Learn more at\n5.7.9  https://support.google.com/mail/?p=InvalidSecondFactor c8sm3769680lfc.46 - gsmtp')

# purchase an adviced stock

This next part needs to be after "login" because I need to check if I have cash to buy the adviced stock

In [9]:
n = inputNumber('\n Which stock (i) would you like to invest in? ')


 Which stock (i) would you like to invest in? 68


In [10]:
price_nok = df[df['i'] == n]['price_nok'].values[0]
symbol = df[df['i'] == n]['symbol'].values[0]
df

,i,symbol,advice,price,price_nok,volume
0,1,ATVI,False,73.08,731.054681,5160900
1,2,ADBE,False,385.26,3853.942616,1687300
2,3,AMD,False,55.17,551.892265,43465700
3,4,ALGN,False,243.74,2438.249424,967900
4,5,ALXN,False,101.50,1015.353723,1064000
...,...,...,...,...,...,...
95,96,WDC,False,42.42,424.347832,3429400
96,97,WDAY,False,168.15,1682.085996,2657300
97,98,WYNN,False,78.25,782.772698,7764900
98,99,XEL,False,60.37,603.910387,2018200


In [11]:
print('Ticker: {} at equivalent: {} NOK'.format(symbol, round(price_nok, 3)))
y = inputNumber('How many stocks do you want to buy? ')
df.loc[df['i'] == n, 'quantity'] = y

Ticker: NFLX at equivalent: 4294.696 NOK
How many stocks do you want to buy? 10


In [12]:
df.loc[df['i'] == n, 'total_nett_price'] = df.loc[df['i'] == n, 'quantity'] * df.loc[df['i'] == n, 'price_nok']
df

,i,symbol,advice,price,price_nok,volume,quantity,total_nett_price
0,1,ATVI,False,73.08,731.054681,5160900,NaN,NaN
1,2,ADBE,False,385.26,3853.942616,1687300,NaN,NaN
2,3,AMD,False,55.17,551.892265,43465700,NaN,NaN
3,4,ALGN,False,243.74,2438.249424,967900,NaN,NaN
4,5,ALXN,False,101.50,1015.353723,1064000,NaN,NaN
...,...,...,...,...,...,...,...,...
95,96,WDC,False,42.42,424.347832,3429400,NaN,NaN
96,97,WDAY,False,168.15,1682.085996,2657300,NaN,NaN
97,98,WYNN,False,78.25,782.772698,7764900,NaN,NaN
98,99,XEL,False,60.37,603.910387,2018200,NaN,NaN


In [13]:
import numpy as np
def bep(price, quantity):
    
    fixed_cost = 79
    variable_cost = 0.001 
    markup = 1.05
    
    cost_purchase = np.where(
                    fixed_cost > variable_cost * quantity * price,
                    fixed_cost, variable_cost * quantity * price)
    
    cost_selling = cost_purchase
    
    total_cost   = cost_purchase + cost_selling
        
    x = Symbol('x') # selling price
    bep = solve(((x - price) * quantity / markup) - total_cost, x)
    bep = bep[0]

    # test assumption that fixed price 
    
    cost_selling = np.where(
                    fixed_cost > variable_cost * quantity * bep,
                    fixed_cost, variable_cost * quantity * bep)
    
    total_cost   = cost_purchase + cost_selling

    bep = solve(((x - price) * quantity / markup) - total_cost, x)
    bep = bep[0]
    
    return bep, total_cost

In [20]:

price = df.loc[df['i'] == n, 'price_nok'].values[0]
print('price: {}'.format(price))

quantity = df.loc[df['i'] == n,'quantity'].values[0]
print('quanitity: {}'.format(quantity))

price = df.loc[df['i'] == n, 'price_nok'].values[0]
print('price: {}'.format(price))


price: 204.971406822204
quanitity: 8.0


In [15]:
quantity = df.loc[df['i'] == n, 'quantity'].values[0]
bep, total_cost = bep(price_nok, quantity)

df.loc[df['i'] == n,'bep'] = bep
df.loc[df['i'] == n,'total_cost'] = total_cost

print('Your break-even point is around: {} nok:'.format(bep))

Your break-even point is around: 4311.28616285548 nok:


In [16]:
price = df.loc[df['i'] == n, 'price'].values[0]
price_nok = df.loc[df['i'] == n, 'price_nok'].values[0]
quantity = df.loc[df['i'] == n, 'quantity'].values[0]
bep = df.loc[df['i'] == n, 'bep'].values[0]
cost = df.loc[df['i'] == n, 'total_cost'].values[0]

In [17]:
print('RECEIPT')
print('---------------------------------------------------')
print('Ticker: {} at {}'.format(symbol, date.today()))
print('Purchase at: {} USD, equiv: {} NOK'.format(round(price, 3), round(price_nok, 3)))
print('Number of stocks: # {} for a cost of {}'.format(quantity, cost))
print('With a bep at: {} NOK'.format(round(bep, 3)))
print('---------------------------------------------------')

RECEIPT
---------------------------------------------------
Ticker: NFLX at 2020-05-24
Purchase at: 429.32 USD, equiv: 4294.696 NOK
Number of stocks: # 10.0 for a cost of 158.0
With a bep at: 4311.286 NOK
---------------------------------------------------
